In [1]:
! pip install vllm -q
from vllm import SamplingParams, LLM

In [2]:
llm = LLM(model="meta-llama/Meta-Llama-3-70B-Instruct", 
          tensor_parallel_size=4)

INFO 07-15 16:34:22 config.py:698] Defaulting to use mp for distributed inference
INFO 07-15 16:34:22 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='meta-llama/Meta-Llama-3-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=meta-llama/Meta-Llama-3-70B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(VllmWorkerProcess pid=126613) INFO 07-15 16:34:23 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=126614) (VllmWorkerProcess pid=126615) INFO 07-15 16:34:23 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 07-15 16:34:23 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 07-15 16:34:24 utils.py:741] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=126615) (VllmWorkerProcess pid=126614) (VllmWorkerProcess pid=126613) INFO 07-15 16:34:24 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 07-15 16:34:24 utils.py:741] Found nccl from library libnccl.so.2
INFO 07-15 16:34:24 utils.py:741] Found nccl from library libnccl.so.2
INFO 07-15 16:34:24 utils.py:741] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=126615) (VllmWorkerProcess pid=126614) INFO 07-15 16:34:24 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 07-15 16:34:24 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=126613) INFO 07-15 16:

In [ ]:
! pip install transformers -q
import re
from transformers import ( 
    AutoTokenizer, 
    AutoModelForSeq2SeqLM
)
from tqdm import tqdm

In [6]:
def summarization(text):
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
    
    input_ids = summary_tokenizer(
        [WHITESPACE_HANDLER(text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = summary_model.generate(
        input_ids=input_ids,
        max_length=1000,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = summary_tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary

In [7]:
def count_words(s):
    return len(s.split())

In [8]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-70B-Instruct')
summary_model_name = "csebuetnlp/mT5_multilingual_XLSum"
summary_tokenizer = AutoTokenizer.from_pretrained(model_name)
summary_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
# start of the story
test_prompt = """
Theodore Nott had been eagerly awaiting his invitation to attend Hogwarts School of Witchcraft and Wizardry from the very beginning. He knew he was special, believed in it. He had always taken pride in his family. Theo grew up as an unassuming, calm, and intelligent child, but with a keen desire to achieve the best, clever enough. And, as the son of the Notts, how could he not end up in Slytherin? When he turned 11, he received the coveted letter to the wizarding school, and in September, after the sorting ceremony, he found himself in the Snake House. He liked everything about it: the aristocratic common room, most of his classmates (except Crabbe and Goyle), the head of the house, and the subjects. Theo developed a great fondness for Potions and Transfiguration, believing that they were essential for life and the future. Any subject could be transformed into what you desired. Pansy Parkinson also hailed from a pure-blood wizarding family. From childhood, she was told to look perfect, know how to flirt with men, and, above all, seek wealth. Pansy was very intelligent herself, but despite her beauty, she had apparently heard many different nicknames in kindergarten and elementary school. The most offensive was "Pug." Well, she didn't look like one; it was just a detail kids used for insults. But in Slytherin at Hogwarts, you wouldn't hear such things. Pansy understood from the first days that Draco Malfoy was obviously the best option for her, both as a friend and in hypothetical relationships. In the future, of course. She loved Ancient Runes and Divination; she enjoyed dueling and exploring aspects of the future. And runes were always interesting to lay out: you never knew what answer they would give, whether it would be Gebo with Wunjo or something else. The years passed, Theo and Pansy grew up. Now it was the fifth year, they were 15, the first exam... Now, instead of "Pug," the girl heard the occasional very quiet "slut," and Theo suddenly began to hear "womanizer," although he just preferred studying, didn't invest much in relationships, because grades for the O.W.L.s were much more important, so it never came to intimacy. So, initially, they judge based on appearance, and then they see an almost emotionless guy. And Pansy just got used to Draco, got used to the fact that he didn't care about her, just occasionally drank, but never allowed herself to kiss someone else or, worse, intimacy. Neither paid much attention to rumors, just lived and continued to prepare. And finally, all O.W.L.s were passed with grades above expectations, and the sixth year began. Theo and Pansy didn't interact much before, but they got along quite well. There was never any misunderstanding between them. And both occasionally noticed each other's successes and good qualities. One day, they both sat on the lawn in front of the castle. There was still an hour before classes; Transfiguration was supposed to be next. Theo had mastered non-verbal spells, so his cup easily turned into either a tangerine or a plate. • How do you manage that? - suddenly asked Parkinson. • She had problems with this Transfiguration spell. So, she definitely needed help, and Theo usually didn't refuse. • It's quite simple if you repeat not only the words and movements but also know what you want to get from the subject in the end. What it should be like, what smell, taste, color, textures... • Everything you say sounds so simple, - the girl sighed, - but I just can't... • Want me to prove that it's really easy? And also, - here Theo smiled, - a bright and beautiful girl, resembling a star, certainly won't mind having a reason to rejoice. Flowers for the lady? • The girl snorted. Clearly, he was flirting, but at the same time, it was beautiful, and he was teaching. • Okay, but the flowers should be worthy of me. • Of course, princess, - the guy looks for the nearest tree branch, finds it, takes it, frowns slightly, while imagining beautiful black orchids that would suit Pansy, - Orchideus, - Theo said, after which the wand from the branch turned into exactly the bouquet Nott wanted. He handed her the bouquet with a smile."""

In [10]:
summary_config = SamplingParams(
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.2, 
    top_k=60, 
    max_tokens=4000,
)

chapter_config = SamplingParams(
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.3, 
    top_k=80, 
    max_tokens=4000,
)

In [11]:
# prompt for generating plan
plan_prompt = f"""
You are a popular fanfiction creator tasked with writing an extended piece for a Harry Potter fanfiction.
You have to come up with an interesting plot. 
Follow the classical composition and include beginning, climax and outcome in this book. Write a detailed plan for each composition chapter. In the format of 
Chapter 1: What happens in this chapter
Chapter 2: What happens in this chapter
...
It is important to display the following aspects in the plot:
Action and Conflict: Introduce conflicts and challenges that the characters must face. Whether it's a battle with dark forces, a personal dilemma, or a complex mystery, ensure there is plenty of action and tension to keep readers hooked.
Magical Elements: Highlight the magical aspects of the Harry Potter universe. Describe new spells, potions, magical creatures, and enchanted locations. Make magic an integral part of the plot and the characters' lives.
World-Building: Expand on the existing lore of the Harry Potter universe. Introduce new locations, traditions, and histories. Make the world feel alive and full of possibilities.
Do not use p.s., p.p.s. and exc.
The following text is the beginning of the first chapter of this book. Generate the summary according to this text. 
"""

In [12]:
messages = [
        {"role": "system", "content": plan_prompt},
        {"role": "user", "content": test_prompt}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=False, 
    add_generation_prompt=True,
)

summary_heading = """
You are a popular fanfiction creator tasked with writing an new chapter for a Harry Potter fanfiction accrding to the summary below.
Your text should be distinct yet cohesive, maintaining the original tone and style of the Harry Potter series.
Instructions:
1. The text should be for secondary school students.
2. Always narrate in the third person.
3. Ensure that text is rich in detail and narrative depth.
4. Avoid including any text outside of the story (e.g., meta comments, thank you notes, or personal addresses).
5. Write the text with no additional comments.
6. Use only English letters and Arabic numerals.
Here is summary of whole text: 
"""
book_summary = llm.generate(input_ids, summary_config)
book_summary = book_summary[0].outputs[0].text
book_summary = summary_heading + book_summary

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.25s/it, est. speed input: 129.26 toks/s, output: 50.36 toks/s]


In [13]:
print(book_summary)


You are a popular fanfiction creator tasked with writing an new chapter for a Harry Potter fanfiction accrding to the summary below.
Your text should be distinct yet cohesive, maintaining the original tone and style of the Harry Potter series.
Instructions:
1. The text should be for secondary school students.
2. Always narrate in the third person.
3. Ensure that text is rich in detail and narrative depth.
4. Avoid including any text outside of the story (e.g., meta comments, thank you notes, or personal addresses).
5. Write the text with no additional comments.
6. Use only English letters and Arabic numerals.
Here is summary of whole text: 
Here is a summary of the story:

**Title:** "Beneath the Surface"

**Composition Plan:**

**Chapters 1-3: Introduction and Setup**

* Chapter 1: Introduce Theodore Nott and Pansy Parkinson, two students in their fifth year at Hogwarts, highlighting their backgrounds, personalities, and academic interests.
* Chapter 2: Show the daily lives of Theo a

In [14]:
pattern = r"(?m)^.*?\b(\d+-\d+)\b.*?$"

chapter_ranges = re.findall(pattern, book_summary)
total_chapters = 0

for range_str in chapter_ranges:
    start_chapter, end_chapter = map(int, range_str.split('-'))
    total_chapters += end_chapter - start_chapter + 1

print(f"The total number of chapters is: {total_chapters}")

The total number of chapters is: 13


In [15]:
start_of_chapter_prompt = """
Start generating the chapter {chapter_n} based on what happened before. 
here's what happened in the whole book so far:
{book_summary}
here's what happened in the previous chapter so far:
{previous_chapter}
Start writing the text with no additional comments.
The structure of the begging is: 
Chapter {chapter_n}. Name of the chapter.
Text of the chapter
"""
continue_chapter_prompt = """
Сontinue generating the chapter {chapter_n} based on what happened before.
here's what happened in the whole book so far:
{book_summary}
here's what happened in the chapter so far:
{full_chapter_context}
Start writing the text with no additional comments.
Do not write chapter and the name of the chapter. Just continue writing the story.
"""
final_of_chapter_prompt = """
Finish generating the chapter {chapter_n} based on what happened before. 
here's what happened in the whole book so far:
{book_summary}
here's what happened in the chapter so far:
{full_chapter_context}
Start writing the text with no additional comments.
Do not write chapter and the name of the chapter. Just finish writing the story.
"""

In [16]:
%%time
book = test_prompt[:]
previous_chapter = test_prompt 
words_per_chapter = 40000//total_chapters
print(words_per_chapter)

for chapter_i in range(1, total_chapters+1):
    is_n_k_words_per_chapter = False
    messages = [
        {"role": "system", "content": start_of_chapter_prompt.format(
            chapter_n=chapter_i, 
            previous_chapter=previous_chapter, 
            book_summary=summarization(book))},
        {"role": "user", "content": book_summary}
    ]

    part_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    chapter = llm.generate(part_prompt, chapter_config)[0].outputs[0].text

    while not is_n_k_words_per_chapter:
        messages = [
            {"role": "system", "content": continue_chapter_prompt.format(
                chapter_n=chapter_i, 
                full_chapter_context=chapter, 
                book_summary=summarization(book))},
            {"role": "user", "content": book_summary}
        ]

        part_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        part_output = llm.generate(part_prompt, chapter_config)[0].outputs[0].text
        chapter += part_output

        if count_words(chapter) > words_per_chapter:
            is_n_k_words_per_chapter = True

    messages = [
        {"role": "system", "content": final_of_chapter_prompt.format(
            chapter_n=chapter_i, 
            full_chapter_context=chapter, 
            book_summary=summarization(book))},
        {"role": "user", "content": book_summary}
    ]

    part_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    part_output = llm.generate(part_prompt, chapter_config)[0].outputs[0].text
    chapter += part_output
    book += chapter
    previous_chapter = chapter[:] 

3076


Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.08s/it, est. speed input: 570.61 toks/s, output: 48.58 toks/s]

CPU times: user 5h 38min 58s, sys: 24min 28s, total: 6h 3min 27s
Wall time: 46min 33s


In [17]:
print(count_words(book))
clean_book = ''.join([char for char in book.encode().decode('unicode_escape') if ord(char) < 128])
print(count_words(clean_book))
with open("llama_3_70b_instruct.txt", "w") as file:
    file.write(clean_book)

48280
48244
